In [1]:
import sys
sys.path.append("../..")
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib
import os
from surgeNN import io
import scipy
import gcsfs
fs = gcsfs.GCSFileSystem() #list stores, stripp zarr from filename, load 

In [2]:
#configure the script
tgs        = ['stavanger-svg-nor-nhs.csv','wick-wic-gbr-bodc.csv','esbjerg-esb-dnk-dmi.csv','immingham-imm-gbr-bodc.csv','den_helder-denhdr-nld-rws.csv', 'fishguard-fis-gbr-bodc.csv',  'brest-822a-fra-uhslc.csv', 'vigo-vigo-esp-ieo.csv',  'alicante_i_outer_harbour-alio-esp-da_mm.csv']
tgnames = ['Stavanger (NOR)','Wick (UK)', 'Esbjerg (DK)','Immingham (UK)','Den Helder (NL)','Fishguard (UK)','Brest (FR)','Vigo (PT)', 'Alicante (SP)']

qnts = np.array([.95,.98,.99,.995]) #quantiles, don't touch

max_timesteps_between_extremes = 3

In [3]:
lstms = io.Output('gs://leap-persistent/timh37/surgeNN_output/nns/performance_modified/lstm')
#lstms = io.Output('/home/jovyan/test_surge_models/results/nns_v2/performance/lstm')
lstms.open_performance_data(tgs)
lstms.data = lstms.data.sel(max_timesteps_between_extremes=max_timesteps_between_extremes).load()

#compute metrics for observations (it,i doesn't matter as long as all models have been run with the same n_t)
observed_thresholds = lstms.data.o.isel(it=0).quantile(np.array([.95,.98,.99,.999]),dim='time')
observed_stds = lstms.observed_stds()

lstms=lstms.data

In [6]:
for tg in lstms.tg:
    num_train_samples = np.sum(np.isfinite(lstms.sel(tg=tg).yhat.isel(it=0,i=0).sel(split='train')*lstms.sel(tg=tg).o.isel(it=0).sel(split='train'))).values
    num_val_samples = np.sum(np.isfinite(lstms.sel(tg=tg).yhat.isel(it=0,i=0).sel(split='val')*lstms.sel(tg=tg).o.isel(it=0).sel(split='val'))).values
    num_test_samples = np.sum(np.isfinite(lstms.sel(tg=tg).yhat.isel(it=0,i=0).sel(split='test')*lstms.sel(tg=tg).o.isel(it=0).sel(split='test'))).values

    num_total_samples = num_train_samples + num_val_samples + num_test_samples
    
    print(tg.values)
    print('total')
    print('train: '+str(num_train_samples)+'['+'{0:.3f}'.format(num_train_samples/num_total_samples)+']'+', val: ' + str(num_val_samples) + '['+'{0:.3f}'.format(num_val_samples/num_total_samples)+']'+', test: '+str(num_test_samples)+'['+'{0:.3f}'.format(num_test_samples/num_total_samples)+']' )
    where_exceedances_99 = (lstms.o>=observed_thresholds.sel(quantile=.99)).sel(tg=tg).isel(it=0)
    where_exceedances_99p9 = (lstms.o>=observed_thresholds.sel(quantile=.999)).sel(tg=tg).isel(it=0)
    
    print('99th %ile: '+str(observed_thresholds.sel(quantile=.99).sel(tg=tg).values))
    print('num exceedances: '+str(where_exceedances_99.sum(dim='time').values))
    print('num filtered exceedances: '+str(((where_exceedances_99) & (where_exceedances_99.rolling(time=1+2*3,center='True').sum()>1)).sum(dim='time').values))
    print('99.9th %ile: '+str(observed_thresholds.sel(quantile=.999).sel(tg=tg).values))
    print('num exceedances: '+str(where_exceedances_99p9.sum(dim='time').values))
    print('num filtered exceedances: '+str(((where_exceedances_99p9) & (where_exceedances_99.rolling(time=1+2*3,center='True').sum()>1)).sum(dim='time').values))
    print('')

stavanger-svg-nor-nhs.csv
total
train: 65094[0.592], val: 23178[0.211], test: 21697[0.197]
99th %ile: [0.35030315 0.35051451 0.35015658]
num exceedances: [651 232 217]
num filtered exceedances: [586 199 190]
99.9th %ile: [0.53453294 0.50878325 0.54609422]
num exceedances: [66 24 22]
num filtered exceedances: [66 24 22]

wick-wic-gbr-bodc.csv
total
train: 58938[0.617], val: 21538[0.225], test: 15094[0.158]
99th %ile: [0.42678118 0.43036838 0.42246776]
num exceedances: [590 216 151]
num filtered exceedances: [526 192 128]
99.9th %ile: [0.62680197 0.62971791 0.6123519 ]
num exceedances: [59 22 16]
num filtered exceedances: [59 22 16]

esbjerg-esb-dnk-dmi.csv
total
train: 62889[0.602], val: 20249[0.194], test: 21364[0.204]
99th %ile: [1.06475585 1.08528754 1.09898876]
num exceedances: [629 203 214]
num filtered exceedances: [572 187 192]
99.9th %ile: [1.84939663 1.68720043 1.71996548]
num exceedances: [63 21 22]
num filtered exceedances: [63 21 22]

immingham-imm-gbr-bodc.csv
total
train: 

In [ ]:
observed_thresholds